In [1]:
import pandas as pd
import pickle
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
import s3fs

In [2]:
#df = pd.read_csv('../data/NYC-Airbnb-2023.csv', low_memory=False)
df = pd.read_csv('s3://mlops-course-project/data/NYC-Airbnb-2023.csv', low_memory=False)

In [9]:
df

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,number_of_reviews_ltm,license
0,2595,Skylit Midtown Castle,2845,Jennifer,Manhattan,Midtown,40.753560,-73.985590,Entire home/apt,150,30,49,2022-06-21,0.30,3,314,1,NaN
1,5121,BlissArtsSpace!,7356,Garon,Brooklyn,Bedford-Stuyvesant,40.685350,-73.955120,Private room,60,30,50,2019-12-02,0.30,2,365,0,NaN
2,5203,Cozy Clean Guest Room - Family Apt,7490,MaryEllen,Manhattan,Upper West Side,40.803800,-73.967510,Private room,75,2,118,2017-07-21,0.72,1,0,0,NaN
3,5178,Large Furnished Room Near B'way,8967,Shunichi,Manhattan,Midtown,40.764570,-73.983170,Private room,68,2,575,2023-02-19,3.41,1,106,52,NaN
4,5136,"Large Sunny Brooklyn Duplex, Patio + Garden",7378,Rebecca,Brooklyn,Sunset Park,40.662650,-73.994540,Entire home/apt,275,60,3,2022-08-10,0.03,1,181,1,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42926,839753193689829909,bright studio in Williamsburg,28057253,Jean,Brooklyn,Williamsburg,40.718976,-73.963985,Entire home/apt,76,7,0,NaN,NaN,1,8,0,NaN
42927,839786573617495393,Room in the heart of LES with Gym& Rooftop BBQ,247439246,Charlene,Manhattan,East Village,40.721703,-73.981473,Private room,32,30,0,NaN,NaN,5,79,0,NaN
42928,839797749155954018,Fantastic 3BD apt in Brooklyn,26349246,Jose,Brooklyn,Bushwick,40.688700,-73.907650,Entire home/apt,127,3,0,NaN,NaN,8,356,0,NaN
42929,839814083143454171,The Coziest Home,48533809,Remmy,Staten Island,Bull's Head,40.616911,-74.164652,Entire home/apt,280,1,0,NaN,NaN,1,87,0,NaN


In [3]:
categorical = ['neighbourhood', 'room_type']
numerical = ['availability_365']
target = ['price']
df = df[categorical + numerical + target]
df[categorical] = df[categorical].astype(str)
df = df[df.price <= df.price.quantile(q=0.95)]

In [4]:
df_train, df_test = train_test_split(df, test_size=0.2)
train_dicts = df_train[categorical + numerical].to_dict(orient='records')

In [5]:
dv = DictVectorizer()
X_train = dv.fit_transform(train_dicts)

y_train = df_train[target]

lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_train)

mean_squared_error(y_train, y_pred, squared=False)

72.7985241983876

In [6]:
tests_dicts = df_test[categorical + numerical].to_dict(orient='records')
X_test = dv.transform(tests_dicts)

y_test = df_test[target]
y_pred = lr.predict(X_test)

mean_squared_error(y_test, y_pred, squared=False)

71.64694852421573

In [12]:
tests_dicts = {'neighbourhood': 'West Village',
  'room_type': 'Private room',
  'availability_365': 0}
type(tests_dicts)

dict

In [11]:
X_test = dv.transform(tests_dicts)

In [13]:
type(X_test)

scipy.sparse._csr.csr_matrix

In [7]:
lr = Lasso(0.01)
lr.fit(X_train, y_train)

y_pred = lr.predict(X_test)

mean_squared_error(y_test, y_pred, squared=False)

73.74938600587691

In [8]:
with open('models/lin_reg.bin', 'wb') as f_out:
    pickle.dump((dv, lr), f_out)